# Data Visualization

[Data-based Storytelling](./index.html)

Daniel Winkler (Institute for Retailing & Data Science)  
Stephan Fally (Department Marketing)

# Visual Channels

In [ ]:
print("HI")

[1] "HI"